In [69]:
# GradientTape
import tensorflow as tf

tf.__version__

'2.19.0'

In [70]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train , x_test = x_train/255.0 , x_test/255.0

x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

x_train.shape


TensorShape([60000, 28, 28, 1])

In [71]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


next(iter(train_ds))[0].shape


TensorShape([32, 28, 28, 1])

In [72]:
# Sequential api vs sub class api
from tensorflow.keras import Model, layers

class My_Model(Model):
    def __init__(self):
        super().__init__()
        self.d1 = layers.Dense(512, activation="relu")
        self.d2 = layers.Dense(10)
        self.conv = layers.Conv2D(32, 2, activation="relu")
        self.flatten = layers.Flatten()
        
    def call(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.d1(x)  
        return self.d2(x)
    
        
        

In [73]:
model = My_Model()

model.summary()

Model: "my__model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [74]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) 

In [75]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
test_loss = tf.keras.metrics.Mean(name="test_loss")

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")


In [76]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        prediction = model(images, training=True)
        loss = loss_func(labels, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, prediction)
    
    
@tf.function
def test_step(images, labels):
    prediction = model(images, training=False)
    loss = loss_func(labels, prediction)
    
    test_loss(loss)
    test_accuracy(labels, prediction)

In [77]:
epochs = 5

for i in range(epochs):
    train_loss.reset_state()
    test_loss.reset_state()
    train_accuracy.reset_state()
    test_accuracy.reset_state()
    
    for images, label in train_ds:
        train_step(images, label)
        
        
    for images, label in test_ds:
        test_step(images, label)
        
    print(f"Epoch  : {i} and train loss : {train_loss.result()} and treainb acc : {train_accuracy.result()} Test loss : {test_loss.result()} and test acc : {test_accuracy.result()}")

Epoch  : 0 and train loss : 0.14188505709171295 and treainb acc : 0.9576166868209839 Test loss : 0.05092818662524223 and test acc : 0.9843833446502686
Epoch  : 1 and train loss : 0.03654635697603226 and treainb acc : 0.9885166883468628 Test loss : 0.013602354563772678 and test acc : 0.9962999820709229


2025-06-04 14:40:24.751548: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch  : 2 and train loss : 0.016005784273147583 and treainb acc : 0.9948166608810425 Test loss : 0.007539497688412666 and test acc : 0.997783362865448
Epoch  : 3 and train loss : 0.011470005847513676 and treainb acc : 0.9961666464805603 Test loss : 0.008013488724827766 and test acc : 0.9972500205039978
Epoch  : 4 and train loss : 0.006525575183331966 and treainb acc : 0.997783362865448 Test loss : 0.002597081009298563 and test acc : 0.9992666840553284
